In [2]:
#Things to do:
#Locally:
# - create new notebook for end-to-end predictions
# - use best of n predictions to generate final
# - look into applying poisson blending
# - Use argparse to take inputs (input filepath, output filepath, best of n predictions, etc)
# - run code on updated models and put it into a formatted python file
# - write documentation and put model diagrams on github



#On server:
# - model neds to be trained on cropped celebA images (use 128 img size then crop to 64)
# - Fine tune model on faces from the dataset

In [41]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import zipfile
import json
import os
import time
import logging
import cv2
import requests
from PIL import Image, ImageDraw, ImageFilter
from facenet_pytorch import MTCNN


parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('--repo-dir', default="c:/Users/James/git/de-identification", help='Path to github repository')
parser.add_argument('--image-path', default="c:/Users/James/git/de-identification/downloaded-data/num-faces/train/image_data/16070.jpg", help='Path to github repository')
parser.add_argument('--save-folder', default="c:/Users/James/git/de-identification/dev-notebooks/16070", help='Folder to save results in')
parser.add_argument('--border-factor', default=0.2, help='Width of border used for context in infilling GAN generation')
parser.add_argument('--progress-images', default=True, help='Save progress images & gif in folder')


args = parser.parse_args("")
repo_dir = args.repo_dir
image_path = args.image_path
save_folder = args.save_folder
border_factor = args.border_factor
progress_images = args.progress_images

if not os.path.exists(save_folder):
    os.makedirs(save_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device.type == "cpu":
    logging.warning(" No GPU detected, using CPU instead.")

mtcnn = MTCNN(keep_all=True, device=device) #face detection model

def generate_boxes(img, threshold=0.7):
    all_boxes, probs, landmarks = mtcnn.detect(img.copy(), landmarks=True)
    if all_boxes is None: return []
    all_boxes = [[int(x) for x in box] for box in all_boxes] 
    #gives box values outside of image, e.g. [-6, 135, 69, 229],
    all_boxes = [[max(0, box[0]), max(0, box[1]), box[2], box[3]] for box in all_boxes]
    boxes = []
    for box, prob in zip(all_boxes, probs):
        # width = box[2] - box[0]
        # height = box[3] - box[1]
        #minimum requirement??
        if prob >= threshold:
            boxes.append(box)
        else:
            print(box, prob)
    return boxes

def draw_boxes(img, boxes, masks=None):
    """Draws boxes (& masks if wanted) on image)"""
    frame_draw = img.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        colour = (255, 0, 0) if len(box) == 4 else box[4]
        draw.rectangle(box[:4], outline=colour, width=3) # box = (x1, y1, x2, y2)

    if masks is not None:
        for mask in masks:
            draw.rectangle(mask[:4], fill=(255,255,255))
    return frame_draw

def crop_face(face, x, y):
    """Turns image into a square by cropping"""
    height, width, _ = face.shape
    if height > width:
        diff = height - width
        top_crop = diff // 2
        bottom_crop = diff - top_crop
        face = face[top_crop:-bottom_crop, :]
        y+=top_crop
    elif width > height:
        diff = width - height
        left_crop = diff // 2
        right_crop = diff - left_crop
        face = face[:, left_crop:-right_crop]
        x+=left_crop

    assert face.shape[0] == face.shape[1], "Face is not square"
    return face, x, y

img = Image.open(image_path)
boxes = generate_boxes(img)
np_img = np.array(img)

squares = []
masks = []
for box in boxes:
    x1, y1, x2, y2 = box
    face = np_img[y1:y2, x1:x2]
    square_face, x, y = crop_face(face, x1, y1)

    #4 lines below are only for masking visualisation
    square_size = square_face.shape[0]
    squares.append([x, y, x+square_size, y+square_size, (0, 255, 0)])
    border = int(square_size * border_factor)
    masks.append([x+border, y+border, x+square_size-border, y+square_size-border])

if progress_images:
    img.save(os.path.join(save_folder, "1-original.jpg"))
    draw_boxes(img, boxes).save(os.path.join(save_folder, "2-boxes.jpg"))
    draw_boxes(img, boxes + squares).save(os.path.join(save_folder, "3-boxes_squares.jpg"))
    draw_boxes(img, boxes + squares, masks).save(os.path.join(save_folder, "4-boxes_squares_masks.jpg"))
    